In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc, f1_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,KFold
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import os
seed = 45
#seed =145
% matplotlib inline

In [2]:
PATH='F:/AV/WNS'
OOF_PATH = 'F:/AV/WNS/oof/stack'
submission_path = 'F:/AV/WNS/submission/stack'
stack_submission_path = 'F:/AV/WNS/submission/stack/final_submit'
train_csv = 'train_catboost.csv'
submit_csv = 'sample_submission_M0L0uXE.csv'

In [3]:
oof_files = os.listdir(OOF_PATH)
removed = [i for i in oof_files if "useless" in i]
# removed = removed+[i for i in oof_files if "Stack" in i]

#oof_files = [i for i in oof_files if i not in removed]

oof_dict = {}
for file in oof_files:
    print(file)
    _ = pd.read_csv(f'{OOF_PATH}/{file}')
    _.columns[0]
    val = (float(_.columns[0][-5:]))
    oof_dict[file] = val

LGBM_Stack_Final_v1_20180916224411.csv
LGBM_Stack_Final_v2_20180916224401.csv
Logreg_Stack_Final_v1_20180916223347.csv
Logreg_Stack_Final_v2_20180916223652_useless.csv
Logreg_Stack_Final_v3_20180916223727.csv
XGB_Stack_Final_v1_20180916224830.csv
XGB_Stack_Final_v2_20180916224757.csv


In [4]:
#sorted(oof_dict.iteritems(), key=lambda (k,v): (v,k))
import operator
sorted_x = sorted(oof_dict.items(), key=operator.itemgetter(1))

In [5]:
ix = [(i[1])>0.5254 for i in sorted_x]

new_dict = {}
for i in range(len(sorted_x)):
    if ix[i]:
        k,v = sorted_x[i]
        new_dict[k] = v

In [6]:
new_dict

{'LGBM_Stack_Final_v1_20180916224411.csv': 0.5256,
 'XGB_Stack_Final_v2_20180916224757.csv': 0.5258,
 'Logreg_Stack_Final_v1_20180916223347.csv': 0.5288,
 'Logreg_Stack_Final_v3_20180916223727.csv': 0.53,
 'Logreg_Stack_Final_v2_20180916223652_useless.csv': 0.5337}

In [7]:
# oof_files = ['catboost_10fld_bagtemp_20180916173347.csv','catboost_10fld_20180916082333.csv',
#              'catboost_5fld_baggingtemp0.85_20180916070735.csv',
#             'xgb_10fld_20180915194445.csv','xgb_10fld_20180915183532.csv',
#              'lightgbm_10fld_20180915124747.csv',
#              'logreg_20180915130548.csv'
#             ]
oof_files = [i for i,j in new_dict.items()]
oof_files
test_files = oof_files
Y_true = pd.read_csv(f'{PATH}/{train_csv}')['is_promoted']
submission = pd.read_csv(f'{PATH}/{submit_csv}')

In [8]:
oof_df = pd.concat([pd.read_csv(f'{OOF_PATH}/{i}') for i in oof_files],axis=1)
test_df = pd.concat([pd.read_csv(f'{submission_path}/{i}').iloc[:,1:] for i in oof_files],axis=1)

In [9]:
Y_true.shape[0] == oof_df.shape[0]

True

In [10]:
Y_true.shape

(54808,)

In [11]:
test_df.shape

(23490, 5)

In [12]:
oof_df.shape

(54808, 5)

In [13]:
oof_df.head()

,LGBM_Stack_Final_v1_0.5256,XGB_Stack_Final_v2_0.5258,Logreg_Stack_Final_v1_0.5288,Logreg_Stack_Final_v3_0.530,Logreg_Stack_Final_v2_0.5337
0,0.190897,0.236511,0.197539,0.180993,0.202062
1,0.017784,0.030795,0.022781,0.026188,0.026736
2,0.022466,0.028695,0.017299,0.021006,0.021421
3,0.008105,0.010988,0.018125,0.021663,0.022251
4,0.001866,0.021478,0.018256,0.021612,0.022410


In [14]:
test_df.columns = oof_df.columns
test_df.head()

,LGBM_Stack_Final_v1_0.5256,XGB_Stack_Final_v2_0.5258,Logreg_Stack_Final_v1_0.5288,Logreg_Stack_Final_v3_0.530,Logreg_Stack_Final_v2_0.5337
0,0.166316,0.164720,0.102971,0.083412,0.081597
1,0.007835,0.016085,0.017797,0.021421,0.021976
2,0.007835,0.016085,0.017575,0.021345,0.021903
3,0.007835,0.016085,0.018097,0.021536,0.022082
4,0.007835,0.016085,0.017992,0.021531,0.022077


In [15]:
def kfold_logreg(train_df,test_df, target,num_folds, optimize=False,stratified = False, debug= False,modelname="lightgbm_0"):
    # Divide in training/validation and test data
    train_df = train_df
    test_df = test_df
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['employee_id','index','logreg_0.3322']]
    print(feats)

    #Grid Search
    if optimize:
        logreg = LogisticRegression()
        #param = {'C':[0.001,0.01,0.1,1,10,100]}
        param = {'C':[5,7.5,10,12.5,15,30]}
        clf = GridSearchCV(logreg,param,scoring='f1',refit=True,cv=num_folds,verbose = 1)
        clf.fit(train_df[feats],target)
        best_c = clf.best_params_['C']
        print('Best F1: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_))
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], target)):
        train_x, train_y = train_df[feats].iloc[train_idx], target[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], target[valid_idx]

        if optimize:
            clf = LogisticRegression(C=best_c)
        else:
            clf = LogisticRegression()

        clf.fit(train_x,train_y)

        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

        print('Fold %2d F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.3).astype(int))))
        print('Fold %2d > 0.2 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.2).astype(int))))
        print('Fold %2d > 0.1 F-score : %.6f' % (n_fold + 1, f1_score(valid_y, (oof_preds[valid_idx]>0.1).astype(int))))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
        
    print('Full > 0.28 f1 score %.6f' % f1_score(target, (oof_preds>0.28).astype(int)))
    print('Full > 0.30 f1 score %.6f' % f1_score(target, (oof_preds>0.29).astype(int)))
    print('Full > 0.32 f1 score %.6f' % f1_score(target, (oof_preds>0.30).astype(int)))
    print('Full > 0.25 f1 score %.6f' % f1_score(target, (oof_preds>0.31).astype(int)))
    print('Full > 0.275 f1 score %.6f' % f1_score(target, (oof_preds>0.32).astype(int)))

    # Write submission file and plot feature importance
    if not debug:
        _ = datetime.now().strftime('%Y%m%d%H%M%S')
        Fname = 'F:/AV/WNS/submission/stack/'+str(modelname)+'_'+str(_)+'.csv'
        submission['is_promoted'] = sub_preds
        submission[['employee_id', 'is_promoted']].to_csv(Fname, index= False)
        oof = pd.DataFrame(oof_preds)
        score = f1_score(target, (oof_preds>0.3).astype(int))
        oof.columns = [modelname+'_'+str(round(score,4))]
        OOF_Fname = 'F:/AV/WNS/oof/stack/'+str(modelname)+'_'+str(_)+'.csv'
        oof.to_csv(OOF_Fname,index=False)
    #display_importances(feature_importance_df)
    return 

In [19]:
import gc
oof = kfold_logreg(oof_df,test_df, Y_true,num_folds=5, optimize=False,stratified = True, debug= True,modelname="Logreg_Stack_Final")

Starting LightGBM. Train shape: (54808, 5), test shape: (23490, 5)
['LGBM_Stack_Final_v1_0.5256', 'XGB_Stack_Final_v2_0.5258', 'Logreg_Stack_Final_v1_0.5288', 'Logreg_Stack_Final_v3_0.530', 'Logreg_Stack_Final_v2_0.5337']
Fold  1 F-score : 0.534624
Fold  1 > 0.2 F-score : 0.502128
Fold  1 > 0.1 F-score : 0.444042
Fold  2 F-score : 0.502269
Fold  2 > 0.2 F-score : 0.497373
Fold  2 > 0.1 F-score : 0.434706
Fold  3 F-score : 0.541278
Fold  3 > 0.2 F-score : 0.522006
Fold  3 > 0.1 F-score : 0.453583
Fold  4 F-score : 0.507163
Fold  4 > 0.2 F-score : 0.496531
Fold  4 > 0.1 F-score : 0.441889
Fold  5 F-score : 0.525753
Fold  5 > 0.2 F-score : 0.480204
Fold  5 > 0.1 F-score : 0.430616
Full > 0.28 f1 score 0.523000
Full > 0.30 f1 score 0.523073
Full > 0.32 f1 score 0.522377
Full > 0.25 f1 score 0.523525
Full > 0.275 f1 score 0.523167


In [17]:
#0.529741